In [ ]:
import numpy as np
import cv2

def local_thresholding(img, window_size):
    imgT = img.copy()
    height, width = img.shape[:2]

    half_window = window_size // 2

    for y in range(half_window, height - half_window):
        for x in range(half_window, width - half_window):
            window = img[y - half_window : y + half_window + 1, x - half_window : x + half_window + 1]
            
            rvl = window.ravel()
            ran = round(max(rvl) - min(rvl))
            counts, bins = np.histogram(rvl, bins=ran)

            variance_list = []

            for T in range(1, len(counts)):
                sum_back = 0 
                mean_sum_back = 0
                for i in range(0, T):
                    sum_back += counts[i]
                    mean_sum_back += counts[i] * bins[i]

                w0 = sum_back / sum(counts)
                mean_back = mean_sum_back / sum_back

                sum_obj = 0
                mean_sum_obj = 0
                for j in range(T, len(counts)):
                    sum_obj += counts[j]
                    mean_sum_obj += counts[j] * bins[j]

                w1 = sum_obj / sum(counts)
                mean_obj = mean_sum_obj / sum_obj

                var_sum_back = 0
                for k in range(0, T):
                    var_sum_back += counts[k] * (bins[k] - mean_back)**2 

                var_back = var_sum_back / sum_back

                var_sum_obj = 0
                for l in range(T, len(counts)):
                    var_sum_obj += counts[l] * (bins[l] - mean_obj)**2 

                var_obj = var_sum_obj / sum_obj

                within_class_varianz = w0 * var_back + w1 * var_obj
                variance_list.append(within_class_varianz)

            minvar = min(variance_list)
            THRESH = round(bins[variance_list.index(minvar)])

            if img[y, x] < THRESH:
                imgT[y, x] = 0
            else:
                imgT[y, x] = 255

    return imgT